In [1]:
import pandas as pd
import re
import html
import numpy as np

### Load and clean original data

In [2]:
#Load data on activating and inhibiting compounds downloaded from BRENDA
df_act = pd.read_excel('C:/Users/Elin/Documents/GitHub/predicting-allostery/datafiles/interactions/Allosteric_interactions_BRENDA.xlsx', sheet_name='Activators', names=['EC', 'Enz', 'Met', 'Org', 'Mode'], usecols='A,B,C,E,I')
df_inh = pd.read_excel('C:/Users/Elin/Documents/GitHub/predicting-allostery/datafiles/interactions/Allosteric_interactions_BRENDA.xlsx', sheet_name='Inhibitors', names=['EC', 'Enz', 'Met', 'Org', 'Mode'], usecols='A,B,C,E,I')

#Remove metabolites named "additional information" and data from viruses, and limit species name to two names
#Takes about 2.5 minutes 
df_act = df_act[~df_act['Org'].str.contains('virus')]
df_act = df_act[~df_act['Met'].str.contains('additional information')]
df_act['Org'] = df_act['Org'].apply(lambda x: ' '.join(x.split()[:2]))

df_inh.Org = df_inh.Org.fillna('')
df_inh = df_inh[~df_inh['Org'].str.contains('virus')]
df_inh = df_inh[~df_inh['Met'].str.contains('additional information')]
df_inh['Org'] = df_inh['Org'].apply(lambda x: ' '.join(x.split()[:2]))

### Integrate ChEBI and InChI Ids

In [3]:
#Load chebi and inchi IDs for the compounds in BRENDA
df_chebi_BRENDA = pd.read_csv('C:/Users/Elin/Documents/GitHub/predicting-allostery/datafiles/support/brenda_compounds.tsv', sep='\t', names=['Met', 'Inchi', 'ChEBI'], header=0)

#Integrate chebi and inchi IDs into BRENDA interaction data
df_act_chebi = pd.merge(df_act, df_chebi_BRENDA[['Met', 'ChEBI', 'Inchi']], how="left", on='Met')
df_inh_chebi = pd.merge(df_inh, df_chebi_BRENDA[['Met', 'ChEBI', 'Inchi']], how="left", on='Met')

### Isolate intracellular compounds

#### Load and clean BiGG data

In [9]:
#Load compound data downloaded from BiGG
df_bigg = pd.read_csv('C:/Users/Elin/Documents/GitHub/predicting-allostery/datafiles/support/bigg_metabolites.csv', sep=';')

# Get one database link per row
df_bigg_split = df_bigg.assign(database_links=df_bigg['database_links'].str.split(';')).explode('database_links')

In [12]:
# Isolate the ChEBI id links from the database_links column in the BiGG dataframe and make a new dataframe of only metabolites with ChEBI id
df_bigg_chebis = df_bigg_split[df_bigg_split['database_links'].str.contains('chebi','CHEBI')]

In [6]:
# Extract the ChEBI id (ChEBI:12345) from the database links
def extract_chebi(link):
    for i in range(1,7):
        if link[-i].isdigit():
            continue
        else:
            index = i+5
            return link[-index:]

In [7]:
# Make list of the ChEBI ids
bigg_chebi_list = df_bigg_chebis['database_links'].apply(lambda x: extract_chebi(x)).tolist()

#### Integrate with BRENDA data

In [8]:
#Make dataframes of interactions with metabolites that are documented in BiGG
df_act_intracellular = df_act_chebi[df_act_chebi['ChEBI'].isin(bigg_chebi_list)]
df_inh_intracellular = df_inh_chebi[df_inh_chebi['ChEBI'].isin(bigg_chebi_list)]

#Those with NaN or '-' (without CHEBI id) were not included in the intracellular group

### Remove inorganic compounds from the intracellular interactions

In [9]:
#Get chemical formula from Inchi id
df_act_intracellular['Inchi'] = df_act_intracellular['Inchi'].str.split('/').str[1]
df_inh_intracellular['Inchi'] = df_inh_intracellular['Inchi'].str.split('/').str[1]

C:\Users\Elin\AppData\Local\Temp\ipykernel_19240\2256910586.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_act_intracellular['Inchi'] = df_act_intracellular['Inchi'].str.split('/').str[1]
C:\Users\Elin\AppData\Local\Temp\ipykernel_19240\2256910586.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inh_intracellular['Inchi'] = df_inh_intracellular['Inchi'].str.split('/').str[1]


In [10]:
#Function for parsing the chemical formula to make dictionary of 'atom: number of atoms'
from warnings import warn


ELEMENT_RE = re.compile(r'(?P<atom>[A-Z][a-z]?)(?P<coeff>\d*)')


def parse_formula(formula):
    """ Convert compound formula from string to dictionary.

    For example, C6H12O6 (glucose) becomes {C:6, H:12, O:6}.

    Args:
        formula (str): compound formula

    Returns:
        dict: formula as a dictionary
    """
    return {atom: (int(coeff) if coeff else 1)
            for atom, coeff in re.findall(ELEMENT_RE, formula)}



In [11]:
#Parse formulas in dataframes
df_act_intracellular['formula'] = df_act_intracellular['Inchi'].apply(lambda x: parse_formula(x))
df_inh_intracellular['formula'] = df_inh_intracellular['Inchi'].apply(lambda x: parse_formula(x))

C:\Users\Elin\AppData\Local\Temp\ipykernel_19240\1234169230.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_act_intracellular['formula'] = df_act_intracellular['Inchi'].apply(lambda x: parse_formula(x))
C:\Users\Elin\AppData\Local\Temp\ipykernel_19240\1234169230.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inh_intracellular['formula'] = df_inh_intracellular['Inchi'].apply(lambda x: parse_formula(x))


In [12]:
#Define function for determining whether a compound is organic

def is_organic(formula):
    if all(key in formula for key in ('C', 'H', 'O')):
        return 1
    return 0

In [13]:
#Apply function to data
df_act_intracellular['organic'] = df_act_intracellular['formula'].apply(lambda x: is_organic(x))
df_inh_intracellular['organic'] = df_inh_intracellular['formula'].apply(lambda x: is_organic(x))

C:\Users\Elin\AppData\Local\Temp\ipykernel_19240\2132318377.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_act_intracellular['organic'] = df_act_intracellular['formula'].apply(lambda x: is_organic(x))
C:\Users\Elin\AppData\Local\Temp\ipykernel_19240\2132318377.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inh_intracellular['organic'] = df_inh_intracellular['formula'].apply(lambda x: is_organic(x))


In [14]:
#Filter out the inorganic compounds and drop duplicates
df_act_intra_org = df_act_intracellular[df_act_intracellular.organic != 0]
df_inh_intra_org = df_inh_intracellular[df_inh_intracellular.organic != 0]

In [15]:
#Drop unnecessary columns and duplicates from data
df_act_intra_org_final = df_act_intra_org.drop(['Inchi', 'formula', 'organic'], axis=1).drop_duplicates()
df_inh_intra_org_final = df_inh_intra_org.drop(['Inchi', 'formula', 'organic'], axis=1).drop_duplicates()

In [28]:
#Make one big file of both activators and inhibitors and save to csv
df_BRENDA_intracellular = pd.concat([df_act_intra_org_final, df_inh_intra_org_final]).reset_index(drop=True)
df_BRENDA_intracellular.to_csv('C:/Users/Elin/Documents/GitHub/predicting-allostery/datafiles/interactions/BRENDA_interactions_intracellular.txt')